In [1]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import spacy

In [ ]:
!pip install -U accelerate
!pip install -U transformers

In [3]:
PATH_TO_RAW_DATA = "/home/anton/Anton/Studium/DHBW /Semester_5/Text_Analysis/repo/Burg-NLP/data/rick_and_morty_master.csv"
PATH_TO_CLEAN_DATA = "/home/anton/Anton/Studium/DHBW /Semester_5/Text_Analysis/repo/Burg-NLP/topic_modeling/data/rick_and_morty_master_clean.csv"

df = pd.read_csv( PATH_TO_CLEAN_DATA , index_col = "Unnamed: 0")
df


,episode no.,speaker,dialouge
0,1,Rick,stumbl drunkenli turn light morty got tocom ju...
1,1,Morty,rub eye rick go
2,1,Rick,got surpris morty
3,1,Morty,middl night talk
4,1,Rick,spill alcohol morty ’s bed come got surpris dr...
...,...,...,...
9613,51,Mr. Poopybutthole:,cuz want love thought felt becom
9614,51,Mr. Poopybutthole:,ever think horrifi peopl love would found tr...
9615,51,Mr. Poopybutthole:,wish
9616,51,Mr. Poopybutthole:,wish brave enough love back know mayb tri


In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

In [65]:
df["speaker"] = df["speaker"].astype("str")
df['speaker'] = df["speaker"].apply(lambda x : x.lower())
df["dialouge"] = df["dialouge"].str.replace("[^\w\s]","" , regex=True)
df["speaker"] = df["speaker"].str.replace("[^\w\s]","" , regex=True)
speakers = ["rick" , "morty" , "jerry" , "beth"]
df = df[df.speaker.isin(speakers)]
df

,episode no.,speaker,dialouge
0,1,rick,stumbl drunkenli turn light morty got tocom ju...
1,1,morty,rub eye rick go
2,1,rick,got surpris morty
3,1,morty,middl night talk
4,1,rick,spill alcohol morty s bed come got surpris dra...
...,...,...,...
9357,49,rick,morty m tri stay humbl right soon guy gone po...
9359,49,rick,bye
9361,49,rick,okay bring
9363,49,rick,excus pick name


In [68]:
multilabel = MultiLabelBinarizer()

labels = multilabel.fit_transform(df['speaker']).astype('float32')

texts = [ str(x).lower() for x in df['dialouge']]
labels


array([[1., 1., 1., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 0., 0.],
       [1., 1., 1., ..., 1., 0., 0.],
       [1., 1., 1., ..., 1., 0., 0.]], dtype=float32)

### Modeling

In [73]:
import torch
from transformers import DistilBertTokenizer, AutoTokenizer
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import AdamW

In [47]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels,
                                                                    test_size=0.2, random_state=42)


In [69]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels[0]))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
# Lets build custom dataset
class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_len=128):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    label = torch.tensor(self.labels[idx])

    encoding = self.tokenizer(text, truncation=True, padding="max_length", max_length=self.max_len, return_tensors='pt')

    return {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': label
    }

In [71]:
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

In [82]:
optimizer = AdamW(model.parameters(), lr=4e-4)

In [83]:
# Training Arguments
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir = './results',
    num_train_epochs=5,
    save_steps=1000,
    save_total_limit=2
)
model.to("cpu")
trainer = Trainer(model=model,
                  args=args,
                  train_dataset=train_dataset,
                  eval_dataset = val_dataset,
                  compute_metrics=compute_metrics)

In [84]:

trainer.train()

Step,Training Loss


KeyboardInterrupt: 